<a href="https://colab.research.google.com/github/dar-tau/nlp-experiments/blob/master/introbert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Initialization

In [ ]:
!pip install transformers datasets
# !pip install simpletransformers

In [ ]:
%cd /content
!mkdir data
%cd /content/data
!wget https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v2.0.json

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from tqdm import tqdm_notebook as tqdm
import json
import os

import re
import torch
from torch.utils.data import Dataset, DataLoader
import datasets

from transformers import AutoModelForQuestionAnswering, AutoTokenizer
from transformers import pipeline
from transformers import AdamW
from transformers import get_linear_schedule_with_warmup

from transformers.data.processors.squad import SquadV2Processor, squad_convert_examples_to_features

In [4]:
device = 'cuda'

def torchTokenize(*args):
  return tokenizer(*args, truncation = True,
                       padding = True, return_tensors = 'pt')


def setModelHooks(model):
  attentionLayerRegex = r'^(.+\.)*layer\.(\d+)\.attention$'
  def _guyAttentionHook(name):
    layerNum = int(re.match(attentionLayerRegex, name).group(2))
    # Assumes there's only one attention per number
    def _myHook(m, inp, outp):
      assert((type(outp) == tuple) and (len(outp) == 1) )
      model.guyData[layerNum] = outp[0]#.argmax(dim = -1)

    return _myHook


  if hasattr(model, 'guyHooks'):
    print("Removing existing hooks!")
    [hook.remove() for hook in model.guyHooks]
  
  model.guyData = {}
  model.guyHooks = [module.register_forward_hook(_guyAttentionHook(name)) for name, module in model.named_modules()
                                                                          if re.match(attentionLayerRegex, name) is not None]

def squad_to_introbert(squad_zipped_example_and_features):
  squad_example, squad_features = squad_zipped_example_and_features
  res = {'start_position': squad_features.start_position, 
         'end_position': squad_features.end_position}
  res.update({'context': squad_example.context_text,
              'question': squad_example.question_text})
  # res.update({k: torch.Tensor(as_dict[k], device = device) for k in ['input_ids', 'attention_mask','token_type_ids'] })
  return res

def dictToDevice(d, device):
  d_ = {}
  for k, v in d.items():
    if isinstance(v, torch.Tensor):
      d_[k] = v.to(device)
    else:
      d_[k] = v
  return d_

In [5]:
class IntrobertDataset(Dataset):
  def __init__(self, srcDataset, func, device = device):
    self.ds = srcDataset
    self.func = func
    self.device = device
    self.isModelSet = False

  def setModel(self, model, nLayers):
    self.model = model
    self.nLayers = nLayers
    self.isModelSet = True 

  def __getitem__(self, i):
    assert(self.isModelSet)
    data = self.func(self.ds[i])
    context = data['context']
    start_position = None
    end_position = None
    inputs = None
    introspection = None
    use_original = self.choose_use_original()
    if use_original: 
      start_position = data['start_position']
      end_position = data['end_position']
      question = data['question']
    else:
        inputs = torchTokenize(context)
        inputs = dictToDevice(inputs, self.device)

        chosenLayer = np.random.choice(self.nLayers)
        question = "what is the most attended word in layer {}?".format(chosenLayer)

        def introspection(model):
          res = model.guyData[chosenLayer].sum(dim = -1).argmax(dim = -1)
          return (res, res)
        
    return {'context': context, 'inputs': inputs, 'start_position' : start_position, 'end_position': end_position,
            'question': question, 'introspection': introspection, 'use_original': use_original}

  def choose_use_original(self):
    return np.random.choice(2) == 0

  def __len__(self):
    return len(self.ds)

## Main

In [6]:
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-cased-distilled-squad")
model = AutoModelForQuestionAnswering.from_pretrained("distilbert-base-cased-distilled-squad", return_dict = True)
model.to(device)
optimizer = AdamW(model.parameters(), lr = 5e-5)
setModelHooks(model)

In [7]:
max_seq_length = 384
doc_stride = 128
max_query_length = 64
total = 10000
squad_examples = SquadV2Processor().get_train_examples("/content/data")[:total]
squad_features = squad_convert_examples_to_features(squad_examples, tokenizer = tokenizer, 
                                   max_seq_length = max_seq_length,
                                   max_query_length = max_query_length,
                                   doc_stride = doc_stride, is_training = True, return_dataset = None)

convert squad examples to features:   0%|          | 0/10000 [00:00<?, ?it/s]/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:1321: FutureWarning: The `max_len` attribute has been deprecated and will be removed in a future version, use `model_max_length` instead.
  FutureWarning,
add example index and unique id: 100%|██████████| 10000/10000 [00:00<00:00, 801051.18it/s]


In [8]:
dataset = IntrobertDataset(list(zip(squad_examples, squad_features)), squad_to_introbert)
dataset.setModel(model, 6)

In [ ]:
n_epochs = 10
num_training_steps = total * n_epochs 
num_warmup_steps = total 
losses = []
scheduler = get_linear_schedule_with_warmup(optimizer,num_warmup_steps, num_training_steps)


model.train()

for e in range(n_epochs):
  losses.append([])
  t = tqdm(dataset, total = total)
  acc_sum1 = 0
  acc_sum2 = 0
  for i, data in enumerate(t):
    if i >= total:
      break    

    model.eval()
    context = data['context']
    inputs = data['inputs']
    question = data['question']
    introspection = data['introspection']
    use_original = data['use_original']
    if not use_original:
      model(**inputs)

    model.train()
    model.zero_grad()
    if use_original:
      start_position = data['start_position']
      end_position = data['end_position']
    else:
      start_position, end_position = introspection(model)
          
    start_position = torch.Tensor([start_position]).to(device).detach().long()
    end_position = torch.Tensor([end_position]).to(device).detach().long()

    inputs = torchTokenize(context, question)
    inputs = dictToDevice(inputs, device)
    outputs = model(**inputs, start_positions = start_position,
                    end_positions = end_position)
    
    loss = outputs.loss
    losses[e].append(loss.item())
  
    acc_sum1 += int((start_position[0] == outputs.start_logits.argmax()).item()) 
    acc_sum2 += int((end_position[0] == outputs.end_logits.argmax()).item())

    acc1 = acc_sum1/i if i!=0 else 0.0
    acc2 = acc_sum2/i if i!=0 else 0.0

    t.set_postfix_str("Loss: {:.2f}, Acc1: {:.2f}, Acc2: {:.2f}".format(loss.item(), acc1, acc2))
    loss.backward()
    optimizer.step()
    scheduler.step()
    

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  if sys.path[0] == '':
